<a href="https://colab.research.google.com/github/suginouchi/lecture/blob/main/pom2_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ex 6-1

In [1]:
!pip install pulp
import pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 36.5 MB/s eta 0:00:00


In [105]:
# 加工時間と使用する機械
# 辞書と通し番号を統一するために，先頭にNoneをつけている．
pt = [None, [40,100,50], [70,40],[70,80]]
mu = [None, [1,2,3],[2,1],[2,3]]
K = [None, 3,2,2]
J = len(K)-1
M = 1000 # Mの値は（big-M法として機能する範囲内で）小さくする方が良い．

problem = pulp.LpProblem('job_shop', sense = pulp.LpMinimize)

# 変数を登録
# s_jkは辞書であり通し番号は1から数える．
s_jk =[None]+ [pulp.LpVariable.dicts('s{}'.format(j), range(1,K[j]+1), lowBound = 0) for j in range(1,J+1)]
x_ikjl = [None]
for i in range(1,J+1):
  x_i  =[None]
  for k in range(1, K[i]+1):
    x_ik=[None]
    for j in range(1, J+1):
      x = pulp.LpVariable.dicts('x{}_{}_{}'.format(i, k, j), range(1,K[j]+1), lowBound = 0, cat=pulp.LpBinary)
      # print(i,k,j, x)
      x_ik.append(x)
    x_i.append(x_ik)
  x_ikjl.append(x_i)
  # print(x_ikjl)
c_max = pulp.LpVariable('c_max', lowBound = 0)

print(s_jk)
# print(x_ikjl)
# print(c_max)

[None, {1: s1_1, 2: s1_2, 3: s1_3}, {1: s2_1, 2: s2_2}, {1: s3_1, 2: s3_2}]


In [106]:
# 目的関数はメイクスパン最小化
problem += c_max

# ---- * ---- * ---- * ----

# ジョブjの工程k+1の作業ができるのは，工程kの終了後
for j in range(1,J+1):
  for k in range(1,K[j]):
    problem += s_jk[j][k] + pt[j][k-1] <=s_jk[j][k+1]

# メイクスパンは，全ジョブの最遅完了時刻
for j in range(1,J+1):
  problem += s_jk[j][K[j]] + pt[j][K[j]-1] <= c_max


# 同時に加工できるジョブはたかだか1つ
for i in range(1, J):
  for k in range(1,K[i]+1):
    for j in range(i+1,J+1):
      for l in range(1, K[j]+1):
        if mu[i][k-1] == mu[j][l-1]:
          problem += pt[j][l-1] - M * x_ikjl[i][k][j][l] <= s_jk[i][k] - s_jk[j][l]
          problem += pt[i][k-1] - M * (1 - x_ikjl[i][k][j][l]) <= s_jk[j][l] - s_jk[i][k]

print(problem)

job_shop:
MINIMIZE
1*c_max + 0
SUBJECT TO
_C1: s1_1 - s1_2 <= -40

_C2: s1_2 - s1_3 <= -100

_C3: s2_1 - s2_2 <= -70

_C4: s3_1 - s3_2 <= -70

_C5: - c_max + s1_3 <= -50

_C6: - c_max + s2_2 <= -40

_C7: - c_max + s3_2 <= -80

_C8: - s1_1 + s2_2 - 1000 x1_1_2_2 <= -40

_C9: s1_1 - s2_2 + 1000 x1_1_2_2 <= 960

_C10: - s1_2 + s2_1 - 1000 x1_2_2_1 <= -70

_C11: s1_2 - s2_1 + 1000 x1_2_2_1 <= 900

_C12: - s1_2 + s3_1 - 1000 x1_2_3_1 <= -70

_C13: s1_2 - s3_1 + 1000 x1_2_3_1 <= 900

_C14: - s1_3 + s3_2 - 1000 x1_3_3_2 <= -80

_C15: s1_3 - s3_2 + 1000 x1_3_3_2 <= 950

_C16: - s2_1 + s3_1 - 1000 x2_1_3_1 <= -70

_C17: s2_1 - s3_1 + 1000 x2_1_3_1 <= 930

VARIABLES
c_max Continuous
s1_1 Continuous
s1_2 Continuous
s1_3 Continuous
s2_1 Continuous
s2_2 Continuous
s3_1 Continuous
s3_2 Continuous
0 <= x1_1_2_2 <= 1 Integer
0 <= x1_2_2_1 <= 1 Integer
0 <= x1_2_3_1 <= 1 Integer
0 <= x1_3_3_2 <= 1 Integer
0 <= x2_1_3_1 <= 1 Integer



In [107]:
status = problem.solve()
print(pulp.LpStatus[status])

print('Cmax', c_max.value())
for j in range(1, J+1):
  for k in range(1, K[j]+1):
    print('st{}_{}'.format(j,k), s_jk[j][k].value())


for i in range(1, J):
  for k in range(1,K[i]+1):
    for j in range(i+1,J+1):
      for l in range(1, K[j]+1):
        if mu[i][k-1] == mu[j][l-1]:
          print(i,k,j,l,  x_ikjl[i][k][j][l].value())

Optimal
Cmax 280.0
st1_1 0.0
st1_2 70.0
st1_3 170.0
st2_1 170.0
st2_2 240.0
st3_1 0.0
st3_2 90.0
1 1 2 2 1.0
1 2 2 1 1.0
1 2 3 1 0.0
1 3 3 2 0.0
2 1 3 1 0.0
